In [ ]:
import sys
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import random_split, DataLoader

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = torchvision.datasets.ImageFolder(
    root='../images',
    transform=preprocess
)
train_dataset, val_dataset = random_split(dataset, [0.8, 0.2])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

resnet50_model = torchvision.models.resnet50(
    weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1
)
resnet50_model.fc = nn.Identity()
for param in resnet50_model.parameters():
    param.requires_grad = False
resnet50_model.eval()

fc_model = nn.Sequential(
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Linear(1024, 1)
)

model = nn.Sequential(
    resnet50_model,
    fc_model
)

optimizer = torch.optim.Adam(fc_model.parameters(), lr=0.001)

print(model)

for X, y in train_dataloader:
    out = model(X)
    print(out.shape)
    break

# --- Device + training additions for this lecture ---

# TODO: Pick the best available device (cuda -> mps -> cpu) and store it in `device`.

# TODO: Move the backbone, the head, and the combined model to `device`.

# TODO: Create a binary loss function (use BCEWithLogitsLoss) and store it as `loss_fn`.

# TODO: Write a training loop over epochs (e.g., range(10)):
#   - For each batch, move X and y to `device`
#   - Make y float and reshape to (-1, 1)
#   - Forward pass, zero_grad, compute loss, backward, optimizer.step
#   - Accumulate a running sum of loss and print it per epoch